In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# PyTorch画像用
import torchvision
import torchvision.transforms as transforms

batchSize = 20
epochNum = 10

input_dim = 28 * 28
hidden_dim = 1000
output_dim = 10

learningRate = 0.01

# データセットの取得
train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    transform=transforms.ToTensor(),
    download=True,
)

test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    transform=transforms.ToTensor(),
    download=True,
)

# DataLoaderの作成
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,

    batch_size=batchSize,
    shuffle=True,
    
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,

    batch_size=1,
    shuffle=False,
    
    num_workers=0
)

class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Encoder, self).__init__()
        self.inputLayer = nn.Linear(input_dim, hidden_dim)
        self.outputLayer = nn.Linear(hidden_dim, output_dim)
  
    def forward(self, x):
        h = torch.sigmoid(self.inputLayer(x))
        o = F.log_softmax(h, dim=1)
        return o

def oneHot(label):
    return torch.eye(output_dim)[label]
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Encoder(input_dim, hidden_dim, output_dim).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

model.train()

for i in range(epochNum):
    for j, (image, label) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(image.view(-1, input_dim))
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        
        if j % 500 == 0:
            print(i, ", ", j, ",", f'loss: {loss: 0.4f}')


c:\users\user\appdata\local\programs\python\python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\users\user\appdata\local\programs\python\python39\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\user\AppData\Local\Programs\Python\Python39\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


0 ,  0 , loss:  6.9103
0 ,  500 , loss:  6.4921
0 ,  1000 , loss:  6.4437
0 ,  1500 , loss:  6.4067
0 ,  2000 , loss:  6.3844
0 ,  2500 , loss:  6.3868
1 ,  0 , loss:  6.3568
1 ,  500 , loss:  6.3508
1 ,  1000 , loss:  6.3449
1 ,  1500 , loss:  6.3264
1 ,  2000 , loss:  6.3065
1 ,  2500 , loss:  6.3212
2 ,  0 , loss:  6.2930
2 ,  500 , loss:  6.2918
2 ,  1000 , loss:  6.2868
2 ,  1500 , loss:  6.2688
2 ,  2000 , loss:  6.2754
2 ,  2500 , loss:  6.2435
3 ,  0 , loss:  6.2378
3 ,  500 , loss:  6.2574
3 ,  1000 , loss:  6.2404
3 ,  1500 , loss:  6.2275
3 ,  2000 , loss:  6.2094
3 ,  2500 , loss:  6.1932
4 ,  0 , loss:  6.1987
4 ,  500 , loss:  6.1871
4 ,  1000 , loss:  6.1805
4 ,  1500 , loss:  6.1733
4 ,  2000 , loss:  6.1718
4 ,  2500 , loss:  6.1506
5 ,  0 , loss:  6.1804
5 ,  500 , loss:  6.1597
5 ,  1000 , loss:  6.1449
5 ,  1500 , loss:  6.1552
5 ,  2000 , loss:  6.1461
5 ,  2500 , loss:  6.1401
6 ,  0 , loss:  6.1522
6 ,  500 , loss:  6.1209
6 ,  1000 , loss:  6.1173
6 ,  1500 , lo

In [2]:
import numpy as np

model.eval()

print("evaluation")

dataNum = 0
correctNum = 0

for image, label in test_loader:
    dataNum = dataNum + 1
    
    with torch.no_grad():
        out = model(image.view(-1, input_dim))

    out_numpy = out.cpu().detach().numpy()[0]
    
    if np.argmax(out_numpy) == label.cpu().detach().numpy()[0]:
        correctNum = correctNum + 1
        
print(correctNum/dataNum)

evaluation
0.6696333333333333
